# Average winds correlated to FLC index
* using pre-processed data from MODEL_FLC

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import datetime

In [2]:
time = "00z"

root = "/home/jturner/false_low_clouds_2023/Model/Time_Lapse/time_"+time+"/"

In [37]:
date = "20230301"

u = xr.open_dataset(root+date+'/u_'+date+'.nc')

u.dims['latitude']

721

In [40]:
def wind_xarray(date, direction):
    
    wind = xr.open_dataset(root+date+'/'+direction+'_'+date+'.nc')
    
#---Northeast US
#     min_lon = -83 + 360
#     min_lat = 50
#     max_lon = -51 + 360
#     max_lat = 26
    
#---Central America
#     min_lon = -102 + 360
#     min_lat = 23
#     max_lon = -81 + 360
#     max_lat = 5

#---Kuroshio Current
#     min_lon = 114
#     min_lat = 53
#     max_lon = 170
#     max_lat = 16

#---South America
    min_lon = -100 + 360
    min_lat = -15
    max_lon = -40 + 360
    max_lat = -58
    
    wind = wind.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon))

    #---Building the xarray dataset
    
    dims = [wind.dims['latitude'], wind.dims['longitude']]
    
    row_meshgrid, col_meshgrid = np.meshgrid(dims[0], dims[1], indexing='ij')

    ds = xr.Dataset(
        data_vars=dict(
            wind=(["x","y"], wind)
        ),
        coords=dict(
            latitude=(["x","y"], row_meshgrid),
            longitude=(["x","y"], col_meshgrid),
        ),
        attrs=dict(description="FLC index in the designated region"),
    )
    
    ds = ds.expand_dims(dim={"time": 1})
    
    return ds

In [41]:
wind_3d_xarray_u = [wind_xarray('20230301', 'u'), wind_xarray('20230302', 'u'),
                 wind_xarray('20230303', 'u')]

dates = [datetime.datetime(2023, 3, 1), datetime.datetime(2023, 3, 2),
         datetime.datetime(2023, 3, 3)]

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['x', 'y'], <xarray.Dataset>
Dimensions:     (latitude: 173, longitude: 241)
Coordinates:
  * latitude    (latitude) float64 -58.0 -57.75 -57.5 ... -15.5 -15.25 -15.0
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
  * longitude   (longitude) float64 260.0 260.2 260.5 ... 319.5 319.8 320.0
    valid_time  datetime64[ns] ...
Data variables:
    u           (latitude, longitude) float32 ...) to Variable.